In [1]:
%run "00_path_config.ipynb"
%run "10_data_objects.ipynb"
%run "20_process_sample_table.ipynb"

import pathlib as pl
import pandas as pd
import collections as col
import sys
import pickle as pck

LOCAL_RUN = True
FORCE_RERUN = False

DATA_SOURCE_CACHE = pl.Path(".cache", "data-source-files.pck")


if LOCAL_RUN:
    PROCESS_DATA_ROOT = PATH_PREFIX.local
else:
    PROCESS_DATA_ROOT = PATH_PREFIX.remote
    

FILES_EXIST_BY_SOURCE_FOLDER = col.defaultdict(list)

if DATA_SOURCE_CACHE.is_file() and not FORCE_RERUN:
    with open(DATA_SOURCE_CACHE, "rb") as dump:
        FILES_EXIST_BY_SOURCE_FOLDER = pck.load(dump)
        
else:
    
    for project, read_type, subfolder in SUB_DATA_FOLDERS:

        search_path = PROCESS_DATA_ROOT.joinpath(DATA_ROOT, PROJECT_DATA_FOLDER, subfolder)
        assert search_path.is_dir()

        sample_listings = list(search_path.glob("**/sample-folder.lst"))

        if not sample_listings:
            sys.stderr.write(f"WARNING - no sample listings file: {search_path}\n\n")
            continue

        for sample_listing in sample_listings:

            with open(sample_listing, "r") as lst:
                for line in lst:
                    if not line.strip():
                        continue
                    sample_folder = PROCESS_DATA_ROOT.joinpath(DATA_ROOT, line.strip())
                    assert sample_folder.is_dir(), f"Missing / no dir: {sample_folder}"

                    sample_files = list(sample_folder.glob("**/*.fastq.gz"))
                    if not sample_files:
                        raise ValueError(f"Empty folder: {sample_folder}")

                    sample_files = [fp for fp in sample_files if "fail" not in fp.name]
                    for fp in sample_files:
                        FILES_EXIST_BY_SOURCE_FOLDER[sample_folder].append(fp)

        with open(DATA_SOURCE_CACHE, "wb") as dump:
            _ = pck.dump(FILES_EXIST_BY_SOURCE_FOLDER, dump)


FILES_EXIST_PER_SAMPLE = col.defaultdict(list)
for dsource, files_in_source in FILES_EXIST_BY_SOURCE_FOLDER.items():
    for fp in files_in_source:
        smpf = DataFile(fp, dsource)
        FILES_EXIST_PER_SAMPLE[smpf.sample].append(smpf)
        assert smpf.sample in SAMPLES or smpf.sample in ["GM19320"], smpf.sample



Skipping over 19320 - not enough data
